In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Timeseries Multivariant Forecasting with LSTM model

This notebook is about forecasting of the timeseries Load data avaialable in Train.csv.
The change in Load is depend on the multiple variables like Temp, Humidity, Wind and Precipitation.

We will train a LSTM model, which will take  hourly records available from 2015 to 2019.
Then, we will predict the future demand for Jan-2020 and Compare with actual data that how model has worked...

In [ ]:
df = pd.read_csv('/kaggle/input/panama-electricity-load-forecasting/train.csv')

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [ ]:
df.head()

In [ ]:
print(df.dtypes)

It seems that there is no NULL values..Great...!!!

In [ ]:
df.isnull().sum()

From above- There are not Categorical dataset only Numerical dataset.
But datetime column data type is object. Needs to be change

In [ ]:
df['datetime']=pd.to_datetime(df['datetime'])

In [ ]:
df.head()

datetime columne is directly can not be used for the Modem building but the we can create a features that repeated in nature any give that to the model so model can identify the pattern related to that features.

In [ ]:
#from datetime import datetime
df['week_day']=df['datetime'].dt.dayofweek
df['date']=df['datetime'].dt.day
df['month']=df['datetime'].dt.month
df['hour']=df['datetime'].dt.hour
df.head()

In [ ]:
#pip install autoviz

In [ ]:
#from autoviz.AutoViz_Class import AutoViz_Class
#AV = AutoViz_Class()
#d = AV.AutoViz(df)

# EDA Observations
Almost all variables are following the Gaussian curve - No need to modify the parameters further
Temp has the Maximum relationship with the required prediction 'nat_demand'
'Holiday_ID' and 'Holiday' have the negative impact on the predictions values. We may neglect them if our model is not performing well.
Average 'nat_demand' on Holidays are less than Normal days.


Let's look at the 'nat_demand' with the short time frame....

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(30, 6))
ddd=df[0:15000]
ddd.groupby('datetime')['nat_demand'].median().plot()

I think we have reasonable insights to create a model. You may further to the EDA to find more insghts if you want..

# Data Preprocessing

In [ ]:
col=['datetime']
new_df= df.drop(columns=col)
df_for_training = new_df.astype(float)
df_for_training.head()

Scaling of data is required....

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)

# The Most Important part...
Now, for this perticular dataset the past values of the records are import to predict the future demand and the Model has to be dynamic to learn from past data and predict the future demand.

- For this, we have to create parts of data having past 2 days (48 hours)of the data to give it the same to the LSTM model to predict the 49th Hour Load prediction.

I hope I clear, if not please ask me in discussion I will clear your doubt. This is the most import part of the this Notebook

In [ ]:
trainX = []
trainY = []
n_future = 1 
n_past = 96
for i in range(n_past, len(df_for_training_scaled) - n_future +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 0])

Convert it to Numpy array....Think why >>>

In [ ]:
trainX, trainY = np.array(trainX), np.array(trainY)
print(trainX.shape)
print(trainY.shape)

# Creating a LSTM Multivariant model 

Now, we have to create a multi stage Nuetral network -LSTM (Long Short Term Memory) moedel. 
The Modem will have the return sequence True to understand the sequncing of the data and identify the pattern. Hope it works the fine.

In [ ]:
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
#model.add(LSTM(32, activation='relu', return_sequences=True))
#model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(24, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainY.shape[1]))

model.compile(optimizer='adam', loss='mse')
model.summary()

# Train the Model

In [ ]:
history = model.fit(trainX, trainY, epochs=50, batch_size=100,verbose=1)
#validation_split=0.2,
plt.plot(history.history['loss'], label='Training loss')
#plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()

Model has performed well on validation data...
Now, make ready the Test data set ready for prediction Jan-2020

In [ ]:
df_test= pd.read_csv('/kaggle/input/panama-electricity-load-forecasting/Test_Jan.csv',encoding="latin-1")
df_test['datetime']=pd.to_datetime(df_test['datetime'])
df_test['week_day']=df_test['datetime'].dt.dayofweek
df_test['date']=df_test['datetime'].dt.day
df_test['month']=df_test['datetime'].dt.month
df_test['hour']=df_test['datetime'].dt.hour
df_for_pred=df_test.drop(columns=col)
df_for_pred.astype(float)
df_pred_scaled=scaler.transform(df_for_pred)

X_pred = []
for i in range(n_past, len(df_pred_scaled) - n_future +1):
    X_pred.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
X_pred= np.array(X_pred)

print('X for prediction shape == {}.'.format(X_pred.shape))

# Make a Prediction

As you can see, values are not in the format in which we required.
We have to do the inverse Transform..

In [ ]:
prediction = model.predict(X_pred) 
prediction_copies = np.repeat(prediction, df_for_training.shape[1], axis=-1)
y_pred_future = scaler.inverse_transform(prediction_copies)[:,0]
y_pred_future
df_forecast=[]
df_forecast = pd.DataFrame(y_pred_future)
temp_df=[]
temp_df=pd.DataFrame(np.zeros(n_past))
df_forecast =pd.concat([temp_df,df_forecast], ignore_index=True)
df_forecast
df_actual= pd.read_csv('/kaggle/input/panama-electricity-load-forecasting/Predict_Jan.csv',encoding="latin-1")
df_actual['Pred_nat_demand']=df_forecast
df_actual = df_actual.iloc[n_past:]
df_actual
a4_dims = (25,8)
fig, ax = plt.subplots(figsize=a4_dims)
sns.lineplot(df_actual['datetime'], df_actual['nat_demand'])
sns.lineplot(df_actual['datetime'], df_actual['Pred_nat_demand'])

In [ ]:
df_forecast
#df_forecast